In [5]:
import pandas as pd
import numpy as np
from pickle import load
import streamlit as st
from PIL import Image
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from matplotlib import pyplot as plt
from numpy import sqrt
from numpy import argmax
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics
from sklearn.metrics import plot_roc_curve
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from pickle import load

# custom def : standardization and prediction
def model_prediction(
    sample_case,
    scaler = joblib.load('scaler60.pkl'),
    model = joblib.load('ua60_model')
):
 
    """
    UA5 type model
    he_usg = Urine specific gravity
    he_uph = Urine pH
    he_ubld = Urine blood
    he_uglu = Urine glucose
    he_upro = Urine protein
    """
    
    # print UA5 and eGFR from input case
  #  print("Urine specific gravity :", sample_case['he_usg'].item())
  #  print("Urine pH :", sample_case['he_uph'].item())
  #  print("Urine blood :", sample_case['he_ubld'].item())
  #  print("Urine glucose :", sample_case['he_uglu'].item())
  #  print("Urine protein :", sample_case['he_upro'].item())
    
    # standardization columns
    std_cols=['age','he_uph','he_usg']    
    # feature extraction from input data UA 
    sample_case_features = sample_case.loc[:,['male', 'he_usg', 'he_uph', 'he_ubld', 'he_uglu', 'he_upro', 'age']]
    sample_case_features[std_cols] = scaler.transform(sample_case_features[std_cols])
    
    # predict probability by model
    prob = model.predict_proba(sample_case_features)[:,1]
    
    # applying different threshold according to proteinuria
    # set threshold by proteinuria
    threshold_nonproteinuria = 0.44
    threshold_proteinuria = 0.77
    
    if sample_case_features['he_upro'].item()>1 :
        pred = prob > threshold_nonproteinuria
        if pred == True :
            print("prediction : eGFR<60, abnormal")
        else:
            print("prediction : eGFR>=60, normal")
    else:
        pred = prob > threshold_proteinuria
        if pred == True :
            print("prediction : eGFR<60, abnormal")
        else:
            print("prediction : eGFR>=60, normal")
            
    return prob

# loading in the model to predict on the data

## xgb models
model60 = joblib.load('ua60_model')
## scaler
scaler60 = joblib.load('scaler60.pkl')
  
def welcome():
    return 'welcome all'
  
# defining the function which will make the prediction using 
# the data which the user inputs
#def prediction(male, he_usg, he_uph, he_ubld, he_uglu, he_upro, age):  
#   
#    prediction = classifier.predict(
#        [[male, he_usg, he_uph, he_ubld, he_uglu, he_upro, age]])
#    print(prediction)
#    return prediction
      
  
# this is the main function in which we define our webpage 
def main():
      # giving the webpage a title
    st.title("eGFR60 prediction")
      
    # here we define some of the front end elements of the web page like 
    # the font and background color, the padding and the text to be displayed
    html_temp = """
    <div style ="background-color:yellow;padding:13px">
    <h1 style ="color:black;text-align:center;">Streamlit eGFR60 classifier ML App </h1>
    </div>
    """
      
    # this line allows us to display the front end aspects we have 
    # defined in the above code
    st.markdown(html_temp, unsafe_allow_html = True)
      
    # the following lines create text boxes in which the user can enter 
    # the data required to make the prediction
    male = st.sidebar.selectbox("male", (1,0))
    he_usg = st.sidebar.slider("urine specific gravity", 1.005, 1.030, 0.005)
    he_uph = st.sidebar.slider("urine pH", 6.5, 9.0, 0.5)
    he_ubld = st.sidebar.slider("urine blood", 0, 5, 1)
    he_uglu = st.sidebar.slider("urine glucose", 0, 5, 1)
    he_upro = st.sidebar.slider("urine protein", 0, 5, 1)
    age = st.sidebar.slider("age", 0, 100, 1)
    
    features = {"male" : male,
               "urine specific gravity" : he_usg,
               "urine pH" : he_uph,
               "urine blood" : he_ubld,
               "urine glucose" : he_uglu,
               "urine protein" : he_upro,
               "age" : age}
    sample_case = pd.DataFrame(features, index=[0])
    
    result =""
      
    # the below line ensures that when the button called 'Predict' is clicked, 
    # the prediction function defined above is called to make the prediction 
    # and store it in the variable result
    if st.button("Predict"):
        result = model_prediction(sample_case)
    st.success('The output is {}'.format(result))
     
if __name__=='__main__':
    main()